In [2]:
import cv2 as cv
import imageio
import numpy as np
import math
import matplotlib.pyplot as plt
from os import listdir
from os import system
import os
import random
import sys
sys.path.append('PythonSIFT/')
from testing_pysift_utils import unpack_octave

In [3]:
sift = cv.xfeatures2d.SIFT_create()

In [4]:
dir_from = '/scratch/image_datasets/imagenet/'
dir_to   = '/scratch/image_datasets/1_for_learned_sift/all/'

In [25]:
files = listdir(dir_from)
files_nr = len(files)

In [26]:
files_nr

75

In [46]:
flickr_fail_image = cv.imread(dir_from + '/n00005787_71', 0) 

In [64]:
counter = 0


duplicate_threshold = 1  # the larger the threshold the closer patches will NOT be considered duplicates
random.shuffle(files)
for i in range(files_nr):

    try:
        image = cv.imread(dir_from + '/' + files[i], 0)
        if len(image.shape) != 2:
            continue
#         image = imageio.imread(dir_from + '/' + files[i])
    except Exception as e:
        print "Failed at", i, ":", e
        continue
    
    if image.shape == flickr_fail_image.shape and image[0].all() == flickr_fail_image[0].all() and image[1].all() == flickr_fail_image[1].all():
        print "Found Flickr fail image at", i
        continue

    keypoints, descriptors = sift.detectAndCompute(image, None)

    keypoints_unique = []
    descriptors_unique = []
    
    for j, keypoint in enumerate(keypoints):
        octave, layer, _ = unpack_octave(keypoint)
        if octave == 0 and layer == 1:
            duplicate = False
            for keypoint_unique in keypoints_unique:
                if abs(keypoint_unique.pt[0] - keypoint.pt[0]) < (keypoint.size / duplicate_threshold) and abs(keypoint_unique.pt[1] - keypoint.pt[1]) < (keypoint.size / duplicate_threshold):
                    duplicate = True
                    break
            if not duplicate:
                keypoints_unique.append(keypoint)
                descriptors_unique.append(descriptors[j])
                
    keypoints = keypoints_unique
    descriptors = np.array(descriptors_unique)
    
    for j, keypoint in enumerate(keypoints):
        octave, layer, _ = unpack_octave(keypoint)
        if octave == 0 and layer == 1:
            patch_centre_x = int(keypoint.pt[1])
            patch_centre_y = int(keypoint.pt[0])
            patch_diameter = int(2 * math.floor(keypoint.size / 2) + 1)  # rounding it to the nearest odd number
            patch_diameter *= 3

            if patch_centre_x - patch_diameter >= 0 and patch_centre_x + patch_diameter + 1 < image.shape[0] and patch_centre_y - patch_diameter >= 0 and patch_centre_y + patch_diameter + 1 < image.shape[1]:
                patch = image[patch_centre_x - patch_diameter: patch_centre_x + patch_diameter + 1,
                              patch_centre_y - patch_diameter: patch_centre_y + patch_diameter + 1]

#                 plt.imshow(patch, cmap="gray")
#                 plt.show()
                
                counter += 1
                
                                
                angle = keypoint.angle
                angle = (255. * angle) / 360
                angle = int(round(angle))
                
                descriptor = np.zeros(129, dtype=np.uint8)
                descriptor[: 128] = descriptors[j][:]
                descriptor[128] = angle
                
                
#                 cv.imwrite(dir_to + '/patches/patch' + str(counter).zfill(6) + '.bmp', patch)
#                 np.save(dir_to + '/descriptors_angles/descr_angle_' + str(counter).zfill(6) + '.npy', descriptor)
                




Failed at 3 : 'NoneType' object has no attribute 'shape'
Failed at 8 : 'NoneType' object has no attribute 'shape'
Failed at 13 : 'NoneType' object has no attribute 'shape'
Failed at 16 : 'NoneType' object has no attribute 'shape'
Found Flickr fail image at 17
Failed at 21 : 'NoneType' object has no attribute 'shape'
Found Flickr fail image at 26
Failed at 27 : 'NoneType' object has no attribute 'shape'
Found Flickr fail image at 30
Failed at 31 : 'NoneType' object has no attribute 'shape'
Failed at 40 : 'NoneType' object has no attribute 'shape'
Failed at 45 : 'NoneType' object has no attribute 'shape'
Failed at 46 : 'NoneType' object has no attribute 'shape'
Failed at 51 : 'NoneType' object has no attribute 'shape'
Found Flickr fail image at 52
Failed at 54 : 'NoneType' object has no attribute 'shape'
Found Flickr fail image at 56
Failed at 58 : 'NoneType' object has no attribute 'shape'
Failed at 59 : 'NoneType' object has no attribute 'shape'
Failed at 61 : 'NoneType' object has no 

In [65]:
counter

4526

In [50]:
counter

5603

In [ ]:
if image.shape == flickr_fail_image.shape and image[0].all() == flickr_fail_image[0].all() and image[1].all() == flickr_fail_image[1].all():
    print "it is a flickr fail image"

In [53]:
#             x = random.randint(0, image.shape[0] - patch_size)
#             y = random.randint(0, image.shape[1] - patch_size)
#             patch = image[x : x + patch_size, y : y + patch_size, :]

#             scipy.misc.imsave(dir_to + '/patch_test_images_inpainting' + str(counter).zfill(5) + '_' +str(j) + '.bmp', patch)
#     #         patch.save(dir_to + '/patch_' + str(i).zfill(5) + '_' +str(j) + '.bmp')
#             print('Saving' + dir_to + '/patch_' + str(counter).zfill(5) + '_' +str(j) + '.bmp')

#     #         imageio.imwrite(dir_to + '/patch_' + str(i).zfill(5) + '_' +str(j) + '.bmp')    